# Graphene/BN

## import & prepare

In [ ]:
!pip install git+https://github.com/nmdl-mizo/interface_master.git@develop
!mkdir -p cif_files
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/C_mp-990448_conventional_standard.cif -o 'cif_files/C_mp-990448_conventional_standard.cif'
!curl https://raw.githubusercontent.com/nmdl-mizo/interface_master/develop/test_files/cif_files/BN_mp-984_conventional_standard.cif -o 'cif_files/BN_mp-984_conventional_standard.cif'

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/nmdl-mizo/interface_master.git (to revision develop) to /tmp/pip-req-build-pi2px8wp
  Running command git clone --filter=blob:none --quiet https://github.com/nmdl-mizo/interface_master.git /tmp/pip-req-build-pi2px8wp
  Resolved https://github.com/nmdl-mizo/interface_master.git to commit 2bdf96ed4835b036557ac18de82b4fa2afdabf24
  Preparing metadata (setup.py) ... done

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   951  100   951    0     0   2926      0 --:--:-- --:--:-- --:--:--  2935


In [2]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, convert_vector_index
from interfacemaster.symmetric_tilt import get_csl_twisted_graphenes
from numpy import array, dot, round, cross, eye, pi, ceil, column_stack, arange
from numpy.linalg import inv, det, norm
from gb_code.csl_generator import print_list
import pandas as pd
from pymatgen.core.structure import Structure

## Define an interface core class

In [3]:
stct_1, stct_2 = list(map(lambda x:Structure.from_file(x), ['cif_files/C_mp-990448_conventional_standard.cif', \
                                                      'cif_files/BN_mp-984_conventional_standard.cif']))

/opt/anaconda3/envs/general/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/anaconda3/envs/general/lib/python3.11/site-packages/pymatgen/io/cif.py:1229: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [4]:
my_interface = core(stct_1, stct_2)

Warning!, this programme will rewrite the POSCAR file in this dir!


## Searching two_dimensional CSL

In [31]:
#combining two specified surfaces
hkl_1 = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_2, my_interface.lattice_2)

In [146]:
#thetas with small sigmas: 10.89, 25.3

In [143]:
my_interface.parse_limit(du = 1e-2, S = 1e-2, sgm1=50, sgm2=50, dd = 1e-2)
#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 10, dtheta = 0.01, exact=False, start = 10.89)

Congrates, we found an appx CSL!

U1 = 
[[-4 -6]
 [ 2 -4]
 [ 0  0]]; sigma_1 = 28

U2 = 
[[-3 -6]
 [ 3 -3]
 [ 0  0]]; sigma_2 = 27

D = 
[[ 1.0005109e+00  5.9280000e-05  0.0000000e+00]
 [-5.9280000e-05  1.0005109e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  1.0000000e+00]]

axis = [ 3.23108364e-16  5.59640103e-16 -5.27676003e+00] ; theta = 10.89



## Get bicrystal

In [144]:
hkl = get_primitive_hkl([0, 0, 1], my_interface.conv_lattice_1, my_interface.lattice_1)
my_interface.compute_bicrystal_two_D(hkl_1, hkl_2, normal_ortho = False, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[ 0 -4 -6]
 [ 0  2 -4]
 [ 1  0  0]]
cell 2:
[[ 0 -3 -6]
 [ 0  3 -3]
 [ 1  0  0]]


In [145]:
layer_displacement = 3.4
my_interface.get_bicrystal(two_D=True,xyz_1 = [1,1,1], xyz_2 = [1,1,1], \
                           dp1 = 9.99905800e+00 - layer_displacement, filetype= 'VASP')

## Sample CNID 

In [30]:
gridsize = 0.3
n1 = int(ceil(norm(my_interface.CNID[:,0])/gridsize))
n2 = int(ceil(norm(my_interface.CNID[:,1])/gridsize))

In [38]:
import shutil
try:
    shutil.rmtree('CNID_inputs')
except:
    print('No existing files')
my_interface.sample_CNID([n1,n2], dp1 = 9.99905800e+00 - layer_displacement)

CNID
[[ 0.42857143 -0.28571429]
 [ 0.14285714 -0.42857143]
 [ 0.          0.        ]]
making 16 files...
completed


Please check the POSCAR file